In [1]:
import matplotlib.pyplot as plt
from pyomo.environ import *
from pyomo.dae import *

import numpy as np
#from scipy.interpolate import interp2d
import pandas as pd 
from itertools import permutations, product, combinations
import idaes
from itertools import permutations, product

from fim_doe import *

import pyomo.common.unittest as unittest

In [2]:
import cvxpy as cp

from reactor_kinetics import *

(CVXPY) Aug 15 06:31:29 PM: Encountered unexpected exception importing solver GUROBI:
ImportError('libgurobi95.so: cannot open shared object file: No such file or directory')
(CVXPY) Aug 15 06:31:29 PM: Encountered unexpected exception importing solver GUROBI:
ImportError('libgurobi95.so: cannot open shared object file: No such file or directory')


## FIM reader

In [3]:
def dataframe_return(data):
    fim1 = []
    fim1.append(data['P1'].to_list())
    fim1.append(data['P2'].to_list())
    fim1.append(data['P3'].to_list())
    fim1.append(data['P4'].to_list())
    
    return fim1 

In [4]:
t_measure = [0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]

#t_measure = [0.125]



def FIM_reader(rank='onebyone'):
    
    FIM_set = []
    
    if rank=='onebyone':
        for idx, i in enumerate(t_measure):
            fim1_data = pd.read_csv('./kinetics_fim/CA_' + str(idx)+'.csv')
            
            fim1 = dataframe_return(fim1_data)
            FIM_set.append(fim1)
            
            fim2_data = pd.read_csv('./kinetics_fim/CB_' + str(idx)+'.csv')
            
            fim2 = dataframe_return(fim2_data)
            FIM_set.append(fim2)
            
            fim3_data = pd.read_csv('./kinetics_fim/CC_' + str(idx)+'.csv')
            
            fim3 = dataframe_return(fim3_data)
            FIM_set.append(fim3)
            
    elif rank=='onealong':
        for idx, i in enumerate(t_measure):
            fim1_data = pd.read_csv('./kinetics_fim/CA_' + str(idx)+'.csv')
            
            fim1 = dataframe_return(fim1_data)
            FIM_set.append(fim1)
            
        for idx, i in enumerate(t_measure):
            fim2_data = pd.read_csv('./kinetics_fim/CB_' + str(idx)+'.csv')
            
            fim2 = dataframe_return(fim2_data)
            FIM_set.append(fim2)
            
        for idx, i in enumerate(t_measure):
            fim3_data = pd.read_csv('./kinetics_fim/CC_' + str(idx)+'.csv')
            
            fim3 = dataframe_return(fim3_data)
            FIM_set.append(fim3)
            
            
    return FIM_set
        
FIM_set = FIM_reader(rank='onealong')
print(np.shape(FIM_set))
print(FIM_set[0][1][1])

(27, 4, 4)
0.0


In [5]:
TotalF = np.zeros((4,4))
for i in range(4):
    for j in range(4):
        TotalF[i,j] = sum(FIM_set[k][i][j] for k in range(27))
        
print(TotalF)

print(np.trace(TotalF))
print(np.linalg.det(TotalF))
print(np.linalg.eigvals(TotalF))

[[  17.66733763   18.74907479  -30.09833141  -76.66335672]
 [  18.74907479   30.78667315  -30.04626402 -124.88329795]
 [ -30.09833141  -30.04626402   52.08798793  123.38481043]
 [ -76.66335672 -124.88329795  123.38481043  514.04755797]]
614.5895566797835
663.4981277559746
[5.87563028e+02 2.64907667e+01 9.71982177e-02 4.38563426e-01]


### Jacobian reader

In [6]:
jaco_info = pd.read_csv('./kinetics_fim/Q.csv')

#print(jaco_info)


Q_stack = []

for i in range(27):
    
    Q_res = [jaco_info['A1'].iloc[i], jaco_info['A2'].iloc[i], jaco_info['E1'].iloc[i], jaco_info['E2'].iloc[i]]
    
    
    Q_stack.append(Q_res)
    
print(Q_stack)

[[0.0, 0.0, 0.0, 0.0], [-1.346695417017152, -1.665334536937735e-13, 2.223379846162832, -1.6653345369377348e-13], [-1.039297035228287, -2.220446049250313e-13, 2.004516103508741, -1.6653345369377348e-13], [-0.7733130998612002, -1.3877787807814457e-13, 1.63790005416059, -1.1102230246251565e-13], [-0.560819154400885, -1.249000902703301e-13, 1.2648388615049688, -6.938893903907228e-14], [-0.3990444064009696, -4.8572257327350605e-14, 0.9415433202331, -2.081668171172168e-14], [-0.2797955547643657, -5.5511151231257833e-14, 0.6830391077798161, -2.081668171172168e-14], [-0.1939053002679436, -3.1225022567582534e-14, 0.486126099753268, -3.4694469519536138e-15], [-0.1331100105811684, -1.3877787807814455e-14, 0.3409191457896289, 1.040834085586084e-14], [0.0, 0.0, 0.0, 0.0], [0.0199157340269318, -1.3752037136890438, -0.0468172929494192, 4.379651748244728], [-0.2728277373608145, -1.38740313152641, 0.1675143115261734, 4.866408693837143], [-0.4952849992631014, -1.4015562993399122, 0.4972483613988609, 5.3

### FIM verificator

In [7]:
#optimize_sol = [ 0, 0, 0, 
#                1, 1, 1, 1, 1, 1,
#                1, 1, 1, 1, 1, 1, 
#                0, 1, 1, 0, 1, 1,
#                0, 1, 1, 0, 1, 1]


SelectF = np.zeros((4,4))
for i in range(4):
    for j in range(4):
        SelectF[i,j] = sum(FIM_set[k][i,j]*yi for k, yi in enumerate(optimize_sol))
        
print(SelectF)

print(np.trace(SelectF))
print(np.linalg.det(SelectF))
print(np.linalg.eigvals(SelectF))

NameError: name 'optimize_sol' is not defined

## Covariance without the initial measurement

In [8]:
jaco_info = pd.read_csv('./kinetics_fim/Q.csv')

#print(jaco_info)


Q_stack = []

for i in range(1,9):
    
    Q_res = [jaco_info['A1'].iloc[i], jaco_info['A2'].iloc[i], jaco_info['E1'].iloc[i], jaco_info['E2'].iloc[i]]
    
    Q_stack.append(Q_res)
    
for i in range(10,18):
    
    Q_res = [jaco_info['A1'].iloc[i], jaco_info['A2'].iloc[i], jaco_info['E1'].iloc[i], jaco_info['E2'].iloc[i]]
    
    Q_stack.append(Q_res)
    
for i in range(19,27):
    
    Q_res = [jaco_info['A1'].iloc[i], jaco_info['A2'].iloc[i], jaco_info['E1'].iloc[i], jaco_info['E2'].iloc[i]]
    
    Q_stack.append(Q_res)
    
print(Q_stack)
    
print(np.shape(Q_stack))

[[-1.346695417017152, -1.665334536937735e-13, 2.223379846162832, -1.6653345369377348e-13], [-1.039297035228287, -2.220446049250313e-13, 2.004516103508741, -1.6653345369377348e-13], [-0.7733130998612002, -1.3877787807814457e-13, 1.63790005416059, -1.1102230246251565e-13], [-0.560819154400885, -1.249000902703301e-13, 1.2648388615049688, -6.938893903907228e-14], [-0.3990444064009696, -4.8572257327350605e-14, 0.9415433202331, -2.081668171172168e-14], [-0.2797955547643657, -5.5511151231257833e-14, 0.6830391077798161, -2.081668171172168e-14], [-0.1939053002679436, -3.1225022567582534e-14, 0.486126099753268, -3.4694469519536138e-15], [-0.1331100105811684, -1.3877787807814455e-14, 0.3409191457896289, 1.040834085586084e-14], [0.0199157340269318, -1.3752037136890438, -0.0468172929494192, 4.379651748244728], [-0.2728277373608145, -1.38740313152641, 0.1675143115261734, 4.866408693837143], [-0.4952849992631014, -1.4015562993399122, 0.4972483613988609, 5.32136224081181], [-0.6436372848857008, -1.409

In [9]:
# variance 
variance = [1, # var_CA^2
            4, # var_CB^2
            8  # var_CC^2
           ]
covariance = [ 0.1, # cov_CA_CB | ti, 
               0.1, # cov_CA_CC | ti, 
               0.5 # cov_CB_CC | ti
]

# construct Sigma
sigma = np.zeros((24, 24))

for i in range(24):
    #for j in range(27):
        
        #if i==j:
    if i<8:
        sigma[i,i]=variance[0]
        sigma[i,i+8]=covariance[0]
        sigma[i,i+16]=covariance[1]
    elif 8<= i < 16:
        sigma[i,i]=variance[1]
        sigma[i,i-8]=covariance[0]
        sigma[i,i+8]=covariance[2]
    elif i>=16:
        sigma[i,i]=variance[2]
        sigma[i,i-16]=covariance[1]
        sigma[i,i-8]=covariance[2]
                
            
# check eigen values to make sure it's PSD 
print(np.linalg.eigvals(sigma))

[0.99566752 8.06331241 3.94102007 8.06331241 8.06331241 3.94102007
 3.94102007 0.99566752 0.99566752 0.99566752 0.99566752 8.06331241
 8.06331241 0.99566752 8.06331241 8.06331241 8.06331241 0.99566752
 0.99566752 3.94102007 3.94102007 3.94102007 3.94102007 3.94102007]


In [10]:
sigma_inv = np.linalg.inv(sigma)

In [14]:
print(np.shape(Q_stack))

Q_matrix = []
for i in range(24):
    Q_matrix.append(np.asarray(Q_stack[i]).reshape((4,1)))
# Q: [CA|t1, ..., CA|tN, CB|ti, ..., CC|ti, ...]

FIM_collection = []

for i in range(8):
    # yCA, CA
    mat = sigma_inv[i,i]*Q_matrix[i]@Q_matrix[i].T
    FIM_collection.append(mat)

for i in range(8):
    # yCA, CB
    FIM_collection.append(sigma_inv[i,i+8]*Q_matrix[i]@Q_matrix[i+8].T)

for i in range(8):
    # YCA, CC
    FIM_collection.append(sigma_inv[i,i+16]*Q_matrix[i]@Q_matrix[i+16].T)

for i in range(8,16): 
    FIM_collection.append(sigma_inv[i,i]*Q_matrix[i]@Q_matrix[i].T)

for i in range(8,16): 
    FIM_collection.append(sigma_inv[i,i-8]*Q_matrix[i]@Q_matrix[i-8].T)

for i in range(8,16): 
    FIM_collection.append(sigma_inv[i,i+8]*Q_matrix[i]@Q_matrix[i+8].T)

for i in range(16, 24): 
    FIM_collection.append(sigma_inv[i,i]*Q_matrix[i]@Q_matrix[i].T)

for i in range(16, 24): 
    FIM_collection.append(sigma_inv[i,i-16]*Q_matrix[i]@Q_matrix[i-16].T)

for i in range(16, 24): 
    FIM_collection.append(sigma_inv[i,i-8]*Q_matrix[i]@Q_matrix[i-8].T)
        

print(np.shape(FIM_collection))

(24, 4)
(72, 4, 4)


In [12]:
def eval_fim(y, cov_y):
     
    fim = sum(ya1*FIM_collection[idx] for idx, ya1 in enumerate(y[0])) \
    + sum(yb1*FIM_collection[idx+24] for idx, yb1 in enumerate(y[1])) \
    + sum(yc1*FIM_collection[idx+48] for idx, yc1 in enumerate(y[2])) \
    + sum(yab1*FIM_collection[idx+8] for idx, yab1 in enumerate(cov_y[0])) \
    + sum(yab1*FIM_collection[idx+32] for idx, yab1 in enumerate(cov_y[0])) \
    + sum(yac1*FIM_collection[idx+16] for idx, yac1 in enumerate(cov_y[1])) \
    + sum(yac1*FIM_collection[idx+56] for idx, yac1 in enumerate(cov_y[1])) \
    + sum(ybc1*FIM_collection[idx+40] for idx, ybc1 in enumerate(cov_y[2])) \
    + sum(ybc1*FIM_collection[idx+64] for idx, ybc1 in enumerate(cov_y[2]))
    
    return fim 

def a_opt(ya, yb, yc, yab, yac, ybc):
    fim = eval_fim(ya, yb, yc, yab, yac, ybc)
    return cp.trace(fim)
    
def d_opt(y, cov_y):
    fim = eval_fim(y, cov_y)
    return cp.log_det(fim)

def e_opt(ya, yb, yc, yab, yac, ybc):
    fim = eval_fim(ya, yb, yc, yab, yac, ybc)
    return -cp.lambda_min(fim)

In [15]:
no_res_unit = 8 
cost = [10, 6, 6]

y = []

for idx in range(3):
    create_y = cp.Variable(no_res_unit, nonneg=True)
    y.append(create_y)
    
cov_y = []
for idx in range(3):
    create_cov = cp.Variable(no_res_unit, nonneg=True)
    cov_y.append(create_cov)
    
p_cons = [sum(cp.sum(y[i]*cost[i]) for i in range(3)) <= 100]
#p_cons = [cp.sum(y[0]*10) + cp.sum(y[1]*6) + cp.sum(y[2]*6) <= 100]

#for i in range(3):
for i in y[0]:
    p_cons += [i<=1]
    
for j in y[1]:
    p_cons += [j<=1]
    
for d in y[2]:
    p_cons += [d<=1]
        
p_cons += [cov_y[0][t] <= y[0][t] for t, idx in enumerate(y[0])]
p_cons += [cov_y[0][t] <= y[1][t] for t, idx in enumerate(y[1])]
p_cons += [cov_y[1][t] <= y[0][t] for t, idx in enumerate(y[0])]
p_cons += [cov_y[1][t] <= y[2][t] for t, idx in enumerate(y[2])]
p_cons += [cov_y[2][t] <= y[1][t] for t, idx in enumerate(y[1])]
p_cons += [cov_y[2][t] <= y[2][t] for t, idx in enumerate(y[2])]

p_cons += [y[0][i]+y[1][i]-1<=cov_y[0][i] for i, idx in enumerate(cov_y[0])]
p_cons += [y[0][i]+y[2][i]-1<=cov_y[1][i] for i, idx in enumerate(cov_y[0])]
p_cons += [y[1][i]+y[2][i]-1<=cov_y[2][i] for i, idx in enumerate(cov_y[1])]

obj = cp.Maximize(d_opt(y, cov_y))
#obj = cp.Maximize(d_opt(y[0], y[1], y[2], cov_y[0], cov_y[1], cov_y[2]))

problem = cp.Problem(obj, p_cons)

In [ ]:
def eval_fim(ya, yb, yc, yab, yac, ybc):
     
    fim = sum(ya1*FIM_collection[idx] for idx, ya1 in enumerate(ya)) \
    + sum(yb1*FIM_collection[idx+24] for idx, yb1 in enumerate(yb)) \
    + sum(yc1*FIM_collection[idx+48] for idx, yc1 in enumerate(yc)) \
    + sum(yab1*FIM_collection[idx+8] for idx, yab1 in enumerate(yab)) \
    + sum(yab1*FIM_collection[idx+32] for idx, yab1 in enumerate(yab)) \
    + sum(yac1*FIM_collection[idx+16] for idx, yac1 in enumerate(yac)) \
    + sum(yac1*FIM_collection[idx+56] for idx, yac1 in enumerate(yac)) \
    + sum(ybc1*FIM_collection[idx+40] for idx, ybc1 in enumerate(ybc)) \
    + sum(ybc1*FIM_collection[idx+64] for idx, ybc1 in enumerate(ybc))
    
    return fim 

def a_opt(ya, yb, yc, yab, yac, ybc):
    fim = eval_fim(ya, yb, yc, yab, yac, ybc)
    return cp.trace(fim)
    
def d_opt(ya, yb, yc, yab, yac, ybc):
    fim = eval_fim(ya, yb, yc, yab, yac, ybc)
    return cp.log_det(fim)

def e_opt(ya, yb, yc, yab, yac, ybc):
    fim = eval_fim(ya, yb, yc, yab, yac, ybc)
    return -cp.lambda_min(fim)

In [ ]:
def eval_fim(y, yab, yac, ybc):
     
    fim = sum(ya1*FIM_collection[idx] for idx, ya1 in enumerate(y[0])) \
    + sum(yb1*FIM_collection[idx+24] for idx, yb1 in enumerate(y[1])) \
    + sum(yc1*FIM_collection[idx+48] for idx, yc1 in enumerate(y[2])) \
    + sum(yab1*FIM_collection[idx+8] for idx, yab1 in enumerate(yab)) \
    + sum(yab1*FIM_collection[idx+32] for idx, yab1 in enumerate(yab)) \
    + sum(yac1*FIM_collection[idx+16] for idx, yac1 in enumerate(yac)) \
    + sum(yac1*FIM_collection[idx+56] for idx, yac1 in enumerate(yac)) \
    + sum(ybc1*FIM_collection[idx+40] for idx, ybc1 in enumerate(ybc)) \
    + sum(ybc1*FIM_collection[idx+64] for idx, ybc1 in enumerate(ybc))
    
    return fim 

def a_opt(ya, yb, yc, yab, yac, ybc):
    fim = eval_fim(ya, yb, yc, yab, yac, ybc)
    return cp.trace(fim)
    
def d_opt(y, yab, yac, ybc):
    fim = eval_fim(ya, yab, yac, ybc)
    return cp.log_det(fim)

def e_opt(ya, yb, yc, yab, yac, ybc):
    fim = eval_fim(ya, yb, yc, yab, yac, ybc)
    return -cp.lambda_min(fim)

In [ ]:
no_res_unit = 8

y = []

for idx in range(3):
    create_y = cp.Variable(no_res_unit, nonneg=True)
    y.append(create_y)

yab = cp.Variable(no_res_unit, nonneg=True)
yac = cp.Variable(no_res_unit, nonneg=True)
ybc = cp.Variable(no_res_unit, nonneg=True)

p_cons = [cp.sum(y[0]*10) + cp.sum(y[1]*6) + cp.sum(y[2]*6) <= 100]

#p_cons = []

for i in y[0]:
    p_cons += [i<=1]
for j in y[1]:
    p_cons += [j<=1]
for d in y[2]:
    p_cons += [d<=1]


#for x in yab:
p_cons += [yab[i] <= y[0][i] for i, idx in enumerate(y[0])]
p_cons += [yab[i] <= y[1][i] for i, idx in enumerate(y[1])]
p_cons += [yac[i] <= y[0][i] for i, idx in enumerate(y[0])]
p_cons += [yac[i] <= y[2][i] for i, idx in enumerate(y[2])]
p_cons += [ybc[i] <= y[1][i] for i, idx in enumerate(y[1])]
p_cons += [ybc[i] <= y[2][i] for i, idx in enumerate(y[2])]

p_cons += [y[0][i]+y[1][i]-1<=yab[i] for i, idx in enumerate(y[0])]
p_cons += [y[0][i]+y[2][i]-1<=yac[i] for i, idx in enumerate(y[0])]
p_cons += [y[1][i]+y[2][i]-1<=ybc[i] for i, idx in enumerate(y[1])]
    
obj = cp.Maximize(d_opt(y, yab, yac, ybc))

problem = cp.Problem(obj, p_cons)


In [16]:
print('First time: Optimal value:', problem.solve(verbose=True))

                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) Aug 15 06:32:39 PM: Your problem has 48 variables, 97 constraints, and 0 parameters.
(CVXPY) Aug 15 06:32:39 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Aug 15 06:32:39 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Aug 15 06:32:39 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Aug 15 06:32:39 PM: Compiling problem (target solver=MOSEK).
(CVXPY) Aug 15 06:32:39 PM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> Co

First time: Optimal value: 1.6329791893816545


In [17]:
print(y[0].value)
print(y[1].value)
print(y[2].value)
print(cov_y[0].value)
print(cov_y[1].value)
print(cov_y[2].value)

[9.99999924e-01 9.99999079e-01 9.99999589e-01 9.99999417e-01
 1.70981939e-06 3.86236151e-07 2.59565296e-07 1.97150417e-07]
[9.99999885e-01 9.99999411e-01 9.99998597e-01 8.89583969e-01
 1.01540520e-05 9.99998820e-01 9.99999509e-01 9.99999703e-01]
[9.99999698e-01 5.87347912e-01 6.02771822e-07 5.44203150e-07
 6.66042025e-07 9.78316894e-07 5.23059498e-01 9.99998975e-01]
[9.99999842e-01 9.99998736e-01 9.99998391e-01 8.89583712e-01
 9.80883811e-07 2.08605635e-07 1.49755242e-07 1.18410512e-07]
[9.99999661e-01 5.87347533e-01 4.51444686e-07 3.65715293e-07
 3.22619058e-07 1.79805181e-07 1.31563200e-07 1.04345416e-07]
[9.99999646e-01 5.87347737e-01 4.16860118e-07 3.29668129e-07
 4.28916827e-07 7.80781775e-07 5.23059344e-01 9.99998863e-01]


## Result reader

In [18]:
time_list = [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]

def result(li1, li2, li3, verbose=True):
    
  
    rep1 = []
    
    for i, res in enumerate(li1):
        if res >= 0.99:
            #print(str(time_list[i]) + ': yes')
            rep1.append(1)            
        elif res <= 0.01:
            #print(str(time_list[i]) + ': no')
            rep1.append(0)
        else: 
            #print(str(time_list[i]) + ': '+str(res))
            rep1.append(res)

    
    rep2 = []
    
    for i, res in enumerate(li2):
        if res >= 0.99:
            #print(str(time_list[i]) + ': yes')
            rep2.append(1)        
        elif res <= 0.01:
            #print(str(time_list[i]) + ': no')
            rep2.append(0)
        else: 
            #print(str(time_list[i]) + ': '+str(res))
            rep2.append(res)
            
    
    rep3 = []
    
    for i, res in enumerate(li3):
        if res >= 0.99:
            #print(str(time_list[i]) + ': yes')
            rep3.append(1)
        elif res <= 0.01:
            #print(str(time_list[i]) + ': no')
            rep3.append(0)
        else: 
            #print(str(time_list[i]) + ': '+str(res))
            rep3.append(res)
            
            
    if verbose:
        print('CA:', rep1)
        print('CB:', rep2)
        print('CC:', rep3)
            
    return rep1, rep2, rep3

yaa, ybb, ycc = result(y[0].value, y[1].value, y[2].value)
yab1, yac1, ybc1 = result(cov_y[0].value, cov_y[1].value, cov_y[2].value, verbose=False)

def check(ya, yb, yc, yab, yac, ybc):
    
    for i in range(8):
        
        if yab[i] != min(ya[i], yb[i]):
            print('constraint violated: yab', i)
            
        if yac[i] != min(ya[i], yc[i]):
            print('constraint violated: yac', i)
            
        if ybc[i] != min(yc[i], yb[i]):
            print('constraint violated: ybc', i)


    
def estimate_cost(li1, li2, li3):
    print('cost:', sum(li1)*10 + sum(li2)*6 + sum(li3)*6)
    
estimate_cost(yaa, ybb, ycc)
#check(yaa, ybb, ycc, yab1, yac1, ybc1)
print(yab1)
print(yac1)
print(ybc1)

CA: [1, 1, 1, 1, 0, 0, 0, 0]
CB: [1, 1, 1, 0.8895839685893835, 0, 1, 1, 1]
CC: [1, 0.5873479124974504, 0, 0, 0, 0, 0.5230594984957986, 1]
cost: 99.99994827749579
[1, 1, 1, 0.8895837123862989, 0, 0, 0, 0]
[1, 0.5873475333485287, 0, 0, 0, 0, 0, 0]
[1, 0.58734773732644, 0, 0, 0, 0, 0.5230593442205211, 1]


In [19]:
fim = compute(yaa, ybb, ycc, yab1, yac1, ybc1)
    
print(fim)
print('trace:', np.trace(fim))
print('det:', np.linalg.det(fim))
print('logdet:', np.log(np.linalg.det(fim)))
print('eig:', np.linalg.eigvals(fim))

NameError: name 'compute' is not defined

## round 

In [ ]:
def compute(ya, yb, yc, yab, yac, ybc):

    #print(FIM_collection[33])

    fim = sum(ya1*FIM_collection[idx] for idx, ya1 in enumerate(ya)) \
        + sum(yb1*FIM_collection[idx+24] for idx, yb1 in enumerate(yb)) \
        + sum(yc1*FIM_collection[idx+48] for idx, yc1 in enumerate(yc)) \
        + sum(yab1*FIM_collection[idx+8] for idx, yab1 in enumerate(yab)) \
        + sum(yab1*FIM_collection[idx+32] for idx, yab1 in enumerate(yab)) \
        + sum(yac1*FIM_collection[idx+16] for idx, yac1 in enumerate(yac)) \
        + sum(yac1*FIM_collection[idx+56] for idx, yac1 in enumerate(yac)) \
        + sum(ybc1*FIM_collection[idx+40] for idx, ybc1 in enumerate(ybc)) \
        + sum(ybc1*FIM_collection[idx+64] for idx, ybc1 in enumerate(ybc))

    return fim



In [ ]:
def generate_cov(la1, lb2, lc3):
    
    lab = []
    
    lac = []
    
    lbc = []
    
    for i in range(8):
        lab.append(min(la1[i], lb2[i]))
        lac.append(min(la1[i], lc3[i]))
        lbc.append(min(lb2[i], lc3[i]))
        
    return lab, lac, lbc

In [ ]:
ya0 = [1, 1, 1, 1, 1, 1, 1, 1]
yb0 = [1, 1, 1, 1, 1, 1, 1, 1]
yc0 = [1, 1, 1, 1, 1, 1, 1, 1]

estimate_cost(ya0, yb0, yc0)
print(sum([sum(ya0), sum(yb0), sum(yc0)]))

yab0, yac0, ybc0 = generate_cov(ya0, yb0, yc0)
print('yab:', yab0)
print('yac:', yac0)
print('ybc:', ybc0)

In [ ]:
fim = compute(ya0, yb0, yc0, yab0, yac0, ybc0)
    
print(fim)
print('trace:', np.trace(fim))
print('det:', np.linalg.det(fim))
print('logdet:', np.log(np.linalg.det(fim)))
print('eig:', np.linalg.eigvals(fim))

## figure 

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np

In [ ]:
budget = np.linspace(50, 180, 14)
print(budget)

no_m = [7,8,9,11,12,14,15,17,18,20,21,22,23,24]

ln_det = [-0.46, 0.1255, 0.452, 0.9326, 1.2987,
         1.6275, 1.7816, 2.0136, 2.2191, 2.3738, 
         2.4847, 2.5439, 2.6023, 2.6324] 


In [ ]:
fig, ax = plt.subplots()

line1, =ax.plot(budget, ln_det, marker='o', color="red", label='D-optimality')
ax.set_xlabel('Budget ($)', fontsize=18)
ax.set_ylabel('log$_e$(D-optimality)', fontsize=18)
plt.xticks(fontsize=16)
ax.tick_params(axis='y', labelsize=14)

ax2 = ax.twinx()
line2, =ax2.plot(budget, no_m, marker='^', color="blue", label='No. of measurements')
ax2.set_ylabel('No. of measurements', fontsize=18)
ax2.tick_params(axis='y', labelsize=14)

#ax.legend(handles=[line1, line2])
plt.show()

fig.savefig('twinx.jpg',
            format='jpeg',
            dpi=200,
            bbox_inches='tight')

### tests

In [ ]:

test = [[1,0,0,5,0,0,9,0,0],
        [0,1,0,0,5,0,0,9,0],
        [0,0,1,0,0,5,0,0,9],
        [5,0,0,4,0,0,6,0,0],
        [0,5,0,0,4,0,0,6,0],
        [0,0,5,0,0,4,0,0,6],
        [9,0,0,6,0,0,4,0,0],
        [0,9,0,0,6,0,0,4,0],
        [0,0,9,0,0,6,0,0,4]]

test1 = np.asarray(test)

In [ ]:
print(np.linalg.inv(test1))